In [ ]:
import numpy as np
import cv2
import os
import pandas as pd

In [ ]:
from sewar.full_ref import mse, rmse, psnr, rmse_sw, uqi, ssim, ergas, scc, rase, sam, msssim, vifp, psnrb 
metrics = {'mse':mse, 'rmse':rmse, 'psnr':psnr, 'rmse_sw':rmse_sw, 'uqi':uqi, 'ssim':ssim, 
    'ergas':ergas, 'scc':scc, 'rase':rase, 'sam': sam, 'msssim': msssim, 'vifp':vifp, 'psnrb':psnrb }

Extract images from video in Python

https://www.geeksforgeeks.org/extract-images-from-video-in-python/

OpenCV - Getting Started with Videos

https://docs.opencv.org/3.4/dd/d43/tutorial_py_video_display.html

In [ ]:
def save_image(video_name, frame, slide, num_frame):
    name = f'./data/{video_name}/frame{slide}-{num_frame}.jpg'
    cv2.imwrite(name, frame)

# Saving images with thresholds

In [ ]:
videos_name = [
    # "Introduction",
    # "Moving from experimentation to production",
    # "Components of Vertex AI",
    "Lab intro_ Training an AutoML video classification model"
    # "Tools to interact with Vertex AI",
    # "Lab intro_ Using an image dataset to train an AutoML model"
]

MSE_SAME = 0.01
MSE_NEW = 4
MSE_VALID = 8
MSSSIM_SAME = 0.998119
WHITE_FRAME = 528757248
BLACK_FRAME = 0
NUM_FRAMES_CHECK_STATIC = 12 

def is_static(val):
    return val < MSE_SAME

def check_bw_frame(frame, idx):
    bw_check = np.sum(frame)
    valid_frame = True

    if(bw_check >= WHITE_FRAME):
        print(f'X Image{currentframe} -> white')
        valid_frame = False
    elif(bw_check <= 0):
        print(f'X Image{currentframe} -> black')
        valid_frame = False

    return valid_frame, bw_check


for video_name in videos_name:

    if not os.path.exists(f'data/{video_name}'):
        os.makedirs(f'data/{video_name}')
    
    video_path = f'/mnt/c/Users/Eduardo/Downloads/{video_name}.mp4'
    cam = cv2.VideoCapture(video_path)

    currentframe = 0
    num_slide = 1

    #### Check if video start static
    _, frame = cam.read()
    currentframe += 1
    last_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    last_image_gray = last_frame_gray

    static_found = False
    frame_step = 3

    while((static_found == False) and (currentframe < NUM_FRAMES_CHECK_STATIC)):
        for  _ in range(frame_step):
            ret, frame = cam.read()
            currentframe += 1

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        similarity = mse(frame_gray, last_frame_gray)

        if(is_static(similarity)):
            valid_image, bw = check_bw_frame(last_image_gray, currentframe)

            if(valid_image):
                print(f'Image{currentframe} -> {bw}')
                save_image(video_name, frame, num_slide, currentframe)
                num_slide += 1

            static_found = True

        last_frame_gray = frame_gray
        last_image_gray = frame_gray
        
    #### end

    frame_step = 1

    while(True):
        
        for  _ in range(frame_step):
            ret, frame = cam.read()
            currentframe += 1
    
        if ret:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            similarity = mse(last_frame_gray, frame_gray)

            if(is_static(similarity)):
                newimg_thresh = mse(frame_gray, last_image_gray)

                if(newimg_thresh > MSE_NEW):
                    valid_image = True

                    if(newimg_thresh < MSE_VALID):
                        double_check = msssim(frame_gray, last_image_gray).real
                        if(double_check >= MSSSIM_SAME):
                            print(f'X Image{num_slide}-{currentframe}-> {double_check:5f} msssim')
                            valid_image = False
                        else:
                            print(f'F Image{num_slide}-{currentframe}-> {double_check:5f} msssim')

                    if(valid_image):
                        valid_image, bw = check_bw_frame(frame_gray, currentframe)

                    if(valid_image):
                        print(f'Image{num_slide}-{currentframe} -> {newimg_thresh:.2f}, {bw}')
                        save_image(video_name, frame, num_slide, currentframe)
                        num_slide += 1
                    
                    last_image_gray = frame_gray
                        
            last_frame_gray = frame_gray
        else:
            break

    cam.release()

## Compare Metrics

* Video: What kinds of problems can it solve_.mp4
* Frame: 4500 (same)
    * mse: 5.704356	
    * msssim: 0.999670
* Frame: 8358 (little diff)
    * mse: 4.548090
    * msssim: 0.997047
* Frame: 8591 (little diff)
    * 4.0318262924382715

Results research
* same_image -> msssim = 1, msssim >= 0.999670
* min_change -> 0.997047

* Infuse your apps with ML
* Image3-403 -> 0.998119 msssim (same)

In [ ]:
video_name = "What kinds of problems can it solve_.mp4"
imgs_idx = [0, 167, 511, 1309, 1383, 1590, 1791, 1896, 2791, 3502, 4466, 4500, 5045, 5978, 6754, 7397, 8035, 8358, 8668, 8866]
# test_metrics = ['mse', 'rmse', 'psnr', 'uqi', 'ergas', 'scc', 'rase', 'sam', 'msssim', 'vifp', 'psnrb']
test_metrics = ['ssim']

cam = cv2.VideoCapture(video_name)
_, last_frame = cam.read()
last_image = last_frame
currentframe = 1

data = {'img': imgs_idx[1:]}

for metric in test_metrics:
    data[metric] = []

while(True):
    ret, frame = cam.read()
  
    if ret:
        if(currentframe in imgs_idx):
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            last_image_gray = cv2.cvtColor(last_image, cv2.COLOR_BGR2GRAY)

            for metric in test_metrics:
                if metric == 'msssim':
                    data[metric].append(metrics[metric](frame_gray, last_image_gray).real)
                else:
                    data[metric].append(metrics[metric](frame_gray, last_image_gray))
            
            last_image = frame
            
        currentframe += 1
        last_frame = frame
    else:
        break

cam.release()

In [ ]:
df = pd.DataFrame(data)

In [ ]:
video_name = "Course introduction.mp4"
imgs_idx = [0, 139, 267, 314, 459, 585, 713, 794, 938, 1500]

cam = cv2.VideoCapture(video_name)
_, last_frame = cam.read()
last_image = last_frame
currentframe = 1

sim_mse = []
newimg_mse = []
sim_msssim = []
newimg_msssim = []

while(True):
    ret, frame = cam.read()
  
    if ret:
        if(currentframe in imgs_idx):
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            last_frame_gray = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
            last_image_gray = cv2.cvtColor(last_image, cv2.COLOR_BGR2GRAY)

            # sim_mse.append(mse(last_frame_gray, frame_gray))
            # newimg_mse.append(mse(frame_gray, last_image_gray))
            sim_msssim.append(msssim(last_frame_gray, frame_gray).real)
            newimg_msssim.append(msssim(frame_gray, last_image_gray).real)
            
            last_image = frame
            
        currentframe += 1
        last_frame = frame
    else:
        break

data = pd.DataFrame({
    'img': imgs_idx[1:],
    # 'sim_mse': sim_mse,
    # 'newimg_mse': newimg_mse,
    'sim_msssim': sim_msssim,
    'newimg_msssim': newimg_msssim
})

cam.release()
data

## Analyzing two pairs of images

In [ ]:
cam = cv2.VideoCapture(video_name)

imgs_idx = [4438, 4500, 8035, 8358]
imgs = {}
currentframe = 0

while(True):
    ret, frame = cam.read()
  
    if ret:
        if(currentframe in imgs_idx):
            imgs[currentframe] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
        currentframe += 1
    else:
        break

cam.release()

In [ ]:
comp1_2 = []
comp3_4 = []

for metr in metrics:
    comp1_2.append(metr(imgs[imgs_idx[0]], imgs[imgs_idx[1]]))
    comp3_4.append(metr(imgs[imgs_idx[2]], imgs[imgs_idx[3]]))

data = pd.DataFrame({'metric': metrics.keys(), 'comp1_2': comp1_2, 'comp3_4':comp3_4})
data['good'] = data['comp1_2'] > data['comp3_4']
data

## Frames similarity

In [ ]:
cam = cv2.VideoCapture("Course introduction.mp4")
_, last_frame = cam.read()

similarity_all = []

while(True):
    ret, frame = cam.read()
  
    if ret:
        similarity_all.append(mse(last_frame, frame))
        last_frame = frame
    else:
        break

cam.release()